#### Aerosandbox Simulation Analysis example

This example shows the basic CITROS data analysis functions and based on the simulations made with Aerosandbox python library.



First of all, we should import all neccesary libs:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from citros_data_analysis import data_access as da
from prettytable import PrettyTable, ALL
import json
from platform import python_version

Now we can connect to the simulation database:

In [5]:
batch_id = '8a5d1a17-f817-415a-920e-53ea8812f8f6'

citros = da.CitrosDB(batch = batch_id)

citros.info().print()

the batch 'aerosandbox_1_3': '8a5d1a17-f817-415a-920e-53ea8812f8f6' exists, but not in the database. Please wait ~10 min and try again
{
}


As you can see in the output above, we've got some information about our simulation run (batch):
1. Size of data
2. Sid information. In case we have more then one sid - this will be shown here.
3. Number and a list of topics. First three topics are build-in system topics, and the last one is our simulation results. 

Let's go further:

In [20]:
citros.sid([0]).info().print()

This shows us more detailed statistics about topics: time stamps, durations, number of messages etc.
Let's inspect our data topic:

In [21]:
citros.topic('/poliastro_simple_orbit/state').sid(0).info().print()

Here we can see that our data topic contains 99 messages, and each one's type is a Float64MultiArray. 
Let's print all raw data:

In [22]:
print(citros.topic('/poliastro_simple_orbit/state').data())

To get all json-data columns for a specific topic, function data() is used. It returns pandas DataFrame. To get a specific columns, pass their labels in a list as an argument:

In [23]:
print(citros.topic('/poliastro_simple_orbit/state').data(['data.data[0]','data.data[1]'])[['data.data[0]', 'data.data[1]']])

Now we can create some plots. Let's plot coordinates along one of the axes (supposing messages come once a second): 

In [24]:
fig1, ax1 = plt.subplots()

citros.time_plot(ax1, 
                 topic_name = '/poliastro_simple_orbit/state', 
                 var_name = 'data.data[0]', 
                 time_step = 1, 
                 sids = [0], 
                 y_label = 'X coords', title_text = 'X coords vs. Time')

To create a 3D plot of our orbit, we can use Matplotlib standart functions:

In [26]:
df = citros.topic('/poliastro_simple_orbit/state').sid([0]).data(['data.data[0]', 'data.data[1]', 'data.data[2]'])



fig, ax = citros.plot_3dgraph(df, 'data.data[0]', 'data.data[1]', 'data.data[2]', 

                 scale = False, title = 'Results', set_x_label='X', set_y_label='Y', set_z_label='Z',

                 legend = True, )



ax.set_box_aspect(aspect=None, zoom=0.9)

ax.view_init(70, 70, 0)



fig.tight_layout() 

As you can see here, we've got only a part of orbit because of given time bounds.

Let's do some simple research. We can use more interesting part of this project - Poliastro atmospheric drag simulation. To perform this, it's necessary to use another batch ID:

In [30]:
batch_id = '07e16ec8-2db5-4b3c-97a3-f988febd80ca'

citros = da.CitrosDB(batch = batch_id)

citros.info().print()

You can see here more than one sids - the simulation batch settings have been adjusted in such a way that each simulation corresponds to a coefficient value from the range [2..3].
We will use this CITROS advantage to analyze the effect of drag coefficient to the orbiting duration:

In [31]:
#create a figure to plot on
fig2, ax2 = plt.subplots()

citros.xy_plot(ax2, 
               topic_name = '/poliastro_atmo_drag/state', 
               var_x_name = 'data.data[0]',
               var_y_name = 'data.data[1]',
               sids = [0,1,2,3,4,5,6,7,8,9], 
               x_label = 'x', y_label = 'y', title_text = 'Orbits')
ax2.set_aspect(1,'datalim')

This plot shows us that the orbits have different duration depending on drag coefficient. You can see the small parts of plots where satellite losses it's altitude and velocity and dives down inside the atmosphere.

Let's investigate how powerfull the drag influence is:

In [33]:
# Defining the list of drag coefficients

c_d_list = [2.1474, 2.5537, 2.5896, 2.1429, 2.9438, 2.4808, 2.4451, 2.0637, 2.7386, 2.8972]



# Setting Dataframe

df = citros.topic('/poliastro_atmo_drag/res').set_order({'sid':'asc'}).data('data.data[0]')

df['drag'] = c_d_list



fig3, ax3 = citros.plot_graph(df, 'drag', 'data.data[0]', '.', title = 'Orbit duration vs drag', set_x_label='Drag coefficient', set_y_label = 'Flight duration until deorbiting, days')

ax3.grid()

The orbit duration decreases from 0.0925 days down to 0.076 days while the drag coefficient rises from 2 to 3. 